In [ ]:
print("hello")

In [ ]:
# # Handle table-like data and matrices
# import numpy as np
# import pandas as pd
# import MySQLdb
# # import sqlalchemy
# # from flask.ext.sqlalchemy import SQLAlchemy

# # Reference:
# # https://support.labs.cognitiveclass.ai/knowledgebase/articles/831621-access-mysql-from-python-notebook-using-mysqldb

# from nltk.tokenize import word_tokenize
# from nltk.stem import PorterStemmer
# from nltk.stem import WordNetLemmatizer

In [ ]:
# #Enter the values for you database connection
# dsn_database = "desklib_dev_db"   # e.g. "MySQLdbtest"
# dsn_hostname = "Localhost"       # e.g.: "mydbinstance.xyz.us-east-1.rds.amazonaws.com"
# dsn_port = 3306                        # e.g. 3306 
# dsn_uid = "root"             # e.g. "user1"
# dsn_pwd = "locus123"            # e.g. "Password123"

In [ ]:
# conn = MySQLdb.connect(host=dsn_hostname, port=dsn_port, user=dsn_uid, passwd=dsn_pwd, db=dsn_database)

In [ ]:
# cursor=conn.cursor()
# cursor.execute("""select title from documents_document;""")
# # cursor.fetchone()
# print("\nShow me the records:\n")
# rows = cursor.fetchall()
# import pprint
# pprint.pprint(rows)

In [ ]:
#reference
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html

# data_train = pd.read_sql("documents_document" , dsn_database)

In [1]:
import textract
import re
import nltk.data
import os
import pandas as pd
import numpy as np
import pickle

from gensim.parsing.preprocessing import strip_punctuation
from nltk import word_tokenize, bigrams, trigrams, FreqDist
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk.util import ngrams
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


# def tokenize(text):
#     tokens = word_tokenize(text)
#     return tokens

# def stem_text(text):
#     ps = PorterStemmer() 
#     word_tokens = tokenize(text)
#     output_arr = []
#     for w in word_tokens: 
#         output_arr.append(ps.stem(w))
    
#     return " ".join(output_arr)

SCORE_THRESHOLD = 4
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-zA-Z .#+_]')
STOPWORDS = set(stopwords.words('english'))
ALLOWED_FORMATS = ['.doc', '.docx', '.pdf', '.ppt', '.pptx', '.odt']  # + ['.png', '.jpg', '.jpeg']
POSSIBLE_KEYWORDS = ('assignment brief', 'Table of contents', 'Rubric', 'name', 'Guidelines', 'project report', 'Conclusion', 'Solution', 'requirement', 'assessment requirement')
FILENAME_KEYWORDS = ('assignment brief', 'assignment', 'solution', 'guidelines', 'Assessment' )
ext_map = []
path = input("Folder Path: ")
files1 = []
features_name = []
features = []

for i in POSSIBLE_KEYWORDS:
    features_name.append("content_"+i)
    
for i in FILENAME_KEYWORDS:
    features_name.append("title_"+i)
    
features_name.append("word_count")
features_name.append("extention")
features_name.append("type")

def get_text(loc):
    try:
        return textract.process(loc).decode("utf-8")
    except textract.exceptions.ExtensionNotSupported as e:
        print(e)
        return ''
    

def get_clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text  # HTML decoding
#     text = text.lower() # lowercase text
    text = text.replace('\n', ' ').replace('\r', '')
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text)  # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)  # delete stopwors from text
    return text


def stem_text(text):
    porter = PorterStemmer()
    token_words=word_tokenize(text)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")

    return "".join(stem_sentence)


def remove_stop_words(words):
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)

    return wordsFiltered

def file_features(file):
    
#     print(os.path.join(root, file))
    try:
        files1.append(os.path.join(root, file))
    except:
        pass

    pre, ext = os.path.splitext(file)
#         print(ext_root)

    # print(stem_text("Programmed programming programmer"))
    # print(text1)
    if ext in ALLOWED_FORMATS:
        try:
            try:
                print(os.path.join(root, file))
                text1 = get_text(os.path.join(root, file))
            except:
                root = ""
                print(os.path.join(root, file))
                text1 = get_text(os.path.join(root, file))
            clean = get_clean_text(text1).lower()
            # cleaner = remove_stop_words(clean)
            text = stem_text(clean)
            tokens = word_tokenize(text)
            # Removing stop words
            tokens = remove_stop_words(tokens)

            word_len = len(tokens)

            # Generating uni, bi and trigrams
            ugs = ngrams(tokens, 1)
            bgs = bigrams(tokens)
            tgs = trigrams(tokens)
            # Dictionary to populate frequencies of all uni, bi and trigram words.
            words_collection = {}
            fdist_ugs = FreqDist(ugs)
            fdist_bgs = FreqDist(bgs)
            fdist_tgs = FreqDist(tgs)

            words_collection.update(fdist_ugs.items())
            words_collection.update(fdist_bgs.items())
            words_collection.update(fdist_tgs.items())

            # for i in words_collection:
            #     print(i, words_collection[i])

            f_list = []
            for i in POSSIBLE_KEYWORDS:
                count = 0
                keyword = strip_punctuation(i)
                keyword = stem_text(keyword)
                tokens = word_tokenize(keyword)
                tokens = remove_stop_words(tokens)
                keyword_tuple = tuple(tokens)
            #     print(keyword_tuple)
            #     for j in words_collection:
            #         if(keyword_tuple == j):
            #             count += 1
                f_list.append(words_collection.get(keyword_tuple, 0))
    #         print(pre)
            for i in FILENAME_KEYWORDS:
    #             print(i)
                if (pre.find(i) != -1):
                    f_list.append(0)
    #                 print ("Contains given substring ")
                else:
                    f_list.append(1)
            f_list.append(word_len)
    #         if ext in ['.doc','.docx','.odt']:
    #             f_list.append(0)
    #         elif ext in ['.ppt','pptx']:
    #             f_list.append(1)
    #         elif ext in ['.pdf']:
    #             f_list.append(2)
            f_list.append(ext)
            try:
                if ext_root == 'questions':
                    f_list.append(0)
                else:
                    f_list.append(1)
            except:
                pass

            features.append(f_list)
        except UnicodeDecodeError:
            pass


Folder Path: files


In [2]:
for (root, dirs, files) in os.walk(path, topdown=False):
    for file in files:
        pre_root, ext_root = root.split('/')
        file_features(file)
    #         print(f_list)
print(features_name)
print(features)

Capstone-Project-Requirement-part-2docx-16298.docx


MissingFileError: The file "Capstone-Project-Requirement-part-2docx-16298.docx" can not be found.
Is this the right path/to/file/you/want/to/extract.docx?

In [ ]:
df = pd.DataFrame(np.array(features), columns=features_name)

In [ ]:
df = df.sample(frac=1)
df

In [ ]:
# for dataset in df:
df['extention'] = df['extention'].replace(['.doc', '.docx','.odt'], '.doc')
df['extention'] = df['extention'].replace(['.ppt', '.pptx'], '.ppt')
df['extention'] = df['extention'].replace('.pdf', '.pdf')

df.loc[ df['word_count'].astype(int) <= 200, 'word_count'] = 0
df.loc[(df['word_count'].astype(int) > 200) & (df['word_count'].astype(int) <= 400), 'word_count'] = 1
df.loc[(df['word_count'].astype(int) > 400) & (df['word_count'].astype(int) <= 650), 'word_count']   = 2
df.loc[ df['word_count'].astype(int) > 650, 'word_count'] = 3
# df['word_count'] = df['word_count'].astype(int)

extention_mapping = {".doc": 0, ".ppt": 1, ".pdf": 2}
df['extention'] = df['extention'].map(extention_mapping)


In [ ]:
df

In [ ]:
X_train, X_test = train_test_split(df, test_size=0.33, random_state=42)
# X_train.shape, X_test.shape
Y_train = X_train["type"]
# pd.crosstab(X_train['extention'], X_train['type'])
X_train = X_train.drop("type", axis=1)
# pd.crosstab(X_test['extention'], X_test['type'])
# X_test  = X_test.drop("type", axis=1).copy()

# df.shape, X_train.shape, Y_train.shape, X_test.shape
# X_train

In [ ]:
X_test  = X_test.drop("type", axis=1).copy()
# X_test

# X_train.shape, Y_train.shape, X_test.shape

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(random_forest, open(filename, 'wb'))



# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
Y_pred = loaded_model.predict(X_test)

loaded_model.score(X_train, Y_train)
acc_random_forest = round(loaded_model.score(X_train, Y_train) * 100, 2)
acc_random_forest
print(Y_pred.size)
print(Y_pred)
print(acc_random_forest)

In [ ]:
file_features('Customer-behaviordocx-15262.docx')

print(features)